In [1]:
import math
import sympy as sp
import numpy as np
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *

In [2]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler 

In [3]:
#Rejection sampling algorithm 
#If parameters chosen as BBC+18, output is 1 with probability 1/rho

def Rej(Z, B, sigma, rho):
    u = np.random.random()
    if u > 1/rho * np.exp((-2*Z.flatten().dot(B.flatten()) + np.linalg.norm(B.flatten())**2)/(2*sigma**2)):
        return 0
    else:
        return 1

In [12]:
#Balance algorithm
#Given a matrix x with coeffs in [0, q], returns the same matrix with coeffs in [-(q-1)/2, (q-1)/2]
def balance(x, q):
    return np.array([n if n <= (q-1)/2 else n-q for n in x.flatten()]).reshape(np.shape(x))

In [13]:
class Gen:
    
    A = None
    S = None
    T = None
    
    def __init__(self, A, S, T):
        self.A = A
        self.S = S
        self.T = T
    

In [14]:
class Prover:
    
    A = None
    S = None
    T = None
    sigma = None
    Y = None
     
    def __init__(self, A, S, T, sigma):
        self.A = A
        self.S = S
        self.T = T
        
        self.Y = []
        self.sigma = sigma
    
    def calculateW(self, n):
        v = self.A.shape[1]
        D = DiscreteGaussianDistributionIntegerSampler(self.sigma)
        self.Y = np.array([D() for _ in range(v*n)]).reshape(v,n)
        W = np.matmul(self.A, self.Y)
        return W
    
    def calculateZ(self, C):
        Z = np.matmul(self.S, C) + self.Y
        return Z

In [15]:
class Verifier:
    
    A = None
    T = None
    W = None
    C = None
    
    def __init__(self, A, T):
        self.A = A
        self.T = T
        self.W = []
        self.C = []
        
    def calculateC(self, W):
        self.W = W
        n = W.shape[1]
        l = self.T.shape[1]
        self.C = np.random.randint(2, size=[l,n])
        return self.C
    
    def verify(self, Z, B, q):
        AZ = np.matmul(self.A, Z)
        TC = np.matmul(self.T, self.C)
        return np.array_equal(AZ%q, (TC + self.W)%q) and np.all(np.linalg.norm(Z, np.inf ,axis=0) <= B)

In [16]:
class ZKP_Protocol:
    
    lamb = None
    
    def __init__(self, lamb):
        
        self.lamb = lamb #Security parameter lambda
        q = sp.nextprime(2**32) #Prime for base field Z_q
        l = 2 #Number of equations
        beta = math.floor(math.sqrt(q/2))
        
        ############# Invent matrices A, S, T ###################
        
        r = 100 #poly(lambda)
        v = 100 #poly(lambda)
        n = self.lamb + 2
        
        A = np.array([Rq.random_element() for _ in range(r*v)]).reshape(r,v)
        S = np.array([Rq.random_element() for _ in range(v*l)]).reshape(v,l)
        T = np.matmul(A, S)

        #########################################################
        
        s = np.linalg.norm(S, 2)
        rho = 3
        sigma = 12/np.log(rho)*s*np.sqrt(l*n)+2**-100
        B = math.sqrt(2*v)*sigma
        print(beta, s, sigma, B)
        
        
        gen = Gen(A, S, T)
        prover = Prover(gen.A, gen.S, gen.T, sigma)
        verifier = Verifier(gen.A, gen.T)
        
        print(A)
        print(S)
        print(T)
        
        abort = True
        num_aborts = 0
        
        while abort:
        
            #Protocol starts
            W = prover.calculateW(n)
        
            C = verifier.calculateC(W)
        
            Z = prover.calculateZ(C)
        
        
            #Rejection sampling
        
            abort = not Rej(Z, np.matmul(S,C), sigma, rho)
            num_aborts += abort
            
            #Verification    
            
        bit = verifier.verify(Z, B, q)
        print(bit)
        print("Times aborted: " + str(num_aborts))

In [68]:
lamb = 128
zkp = ZKP_Protocol(lamb)

TypeError: No loop matching the specified signature and casting
was found for ufunc svd_n

In [34]:
def rejSamplingTest():
    res = 0
    r = 100
    n = 10
    rho = 3
    times = 100
    print("Should abort " + str(100/rho) + "% of the time")
    for _ in range (times):
        B = np.random.randint(0, 100, size = [r,n])
        sigma = 12/math.log(rho) * np.linalg.norm(B,2)
        D = DiscreteGaussianDistributionIntegerSampler(sigma)
        Y = np.array([D() for _ in range(r*n)]).reshape(r,n)
        res += Rej(Y+B, B, sigma, rho)
    print("Aborted " + str(100*res/times) + "% of the time")

In [35]:
rejSamplingTest()

Should abort 33.333333333333336% of the time
Aborted 35.0% of the time


In [96]:
n = 4
q = 7
P, x = PolynomialRing(ZZ, 'x').objgen()
Pq, x = PolynomialRing(Integers(q), 'x').objgen()
R = P.quotient(x**n + 1, 'x')
Rq = Pq.quotient(x**n + 1, 'x')
R, Rq

(Univariate Quotient Polynomial Ring in x over Integer Ring with modulus x^4 + 1,
 Univariate Quotient Polynomial Ring in x over Ring of integers modulo 7 with modulus x^4 + 1)

In [121]:
a = P.random_element(degree = 5)
a, (a%(x**n+1)).coefficients(), a.norm(infinity), 

(x^5 - x^4 - x^3 - 2*x^2 + x - 1, [5, 6], 2.00000000000000)

In [131]:
def R(p):
    return p%(p.variables()[0]**n+1)

In [135]:
def Rq(p):
    return R(p)%q

In [10]:
def vector_norm(v):
    return np.sqrt(np.sum(np.array([int(a.norm()) for a in v])))

In [11]:
vector_norm([Rq([6,6,6,6])]*10)

3.1622776601683795

In [153]:
def matrix_norm(A):
    return np.sqrt(sum(np.array([vector_norm(v) for v in A])))

In [157]:
matrix_norm(A)

7.788319211310561